# NLP TASK PART 2 
This document containes 

PART B) Sentence Similarity

Bonus Part A) Fine tuning models


## Installing and importing

In [ ]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
import numpy as np
# import pandas as pd
import spacy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from torch.utils.data import DataLoader
from datasets import load_dataset
from sklearn.metrics import classification_report
import keras
from keras.models import load_model

## Sentence Embeddings

### Preproc


In [ ]:

dataset = load_dataset("paws", "labeled_final")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:

train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']
train_df = train_dataset.to_pandas()
validation_df = validation_dataset.to_pandas()
test_df = test_dataset.to_pandas()

train_size = 7000
validate_size = 1000
test_size = 2000

train_df = train_df[:train_size]
validation_df = validation_df[:validate_size]
test_df = test_df[:test_size]

print(train_df)

        id                                          sentence1  \
0        1  In Paris , in October 1560 , he secretly met t...   
1        2  The NBA season of 1975 -- 76 was the 30th seas...   
2        3  There are also specific discussions , public p...   
3        4  When comparable rates of flow can be maintaine...   
4        5  It is the seat of Zerendi District in Akmola R...   
...    ...                                                ...   
6995  6996  Ayeza Khan ( born 15 January 1991 as Aiza Khan...   
6996  6997  When Francis II died , France withdrew from Sc...   
6997  6998  In codice _ 4 the second file is codice _ 8 an...   
6998  6999  Here , two more sons were born : Louis in 1882...   
6999  7000  Eupithecia yunnani is a moth in the family Geo...   

                                              sentence2  label  
0     In October 1560 , he secretly met with the Eng...      0  
1     The 1975 -- 76 season of the National Basketba...      1  
2     There are also pub

In [ ]:

!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 722.2 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


### Model

In [ ]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1)* np.linalg.norm(vec2))
def add_preprocessing(df):
    phrase1_embeddings = []
    phrase2_embeddings = []
    y = []
    for index, row in df.iterrows():
        phrase1_vec = sum(nlp(token).vector for token in row['sentence1'].split(' '))
        phrase2_vec = sum(nlp(token).vector for token in row['sentence2'].split(' '))
        norm_phrase1_vec = phrase1_vec / np.linalg.norm(phrase1_vec) if np.linalg.norm(phrase1_vec) != 0 else np.zeros(300)
        norm_phrase2_vec = phrase2_vec / np.linalg.norm(phrase2_vec) if np.linalg.norm(phrase2_vec) != 0 else np.zeros(300)
        sim = cosine_similarity(norm_phrase1_vec, norm_phrase2_vec)

        phrase1_embeddings.append(norm_phrase1_vec)
        phrase2_embeddings.append(norm_phrase2_vec)
        y.append(row['label'])

    return np.array(phrase1_embeddings), np.array(phrase2_embeddings), np.array(y)

s1_train, s2_train, yfcnn_train = add_preprocessing(train_df)
s1_test, s2_test, yfcnn_test = add_preprocessing(test_df)
s1_validate, s2_validate, yfcnn_validate = add_preprocessing(validation_df)

In [ ]:


def create_binary_classification_model(embedding_dim, fcnn_units, dropout_rate, l2_lambda):
    input_phrase1 = Input(shape=(embedding_dim,))
    input_phrase2 = Input(shape=(embedding_dim,))
    fcnn_en1 = Dense(fcnn_units, activation='relu', kernel_regularizer=l2(l2_lambda))(input_phrase1)
    fcnn_en2 = Dense(fcnn_units, activation='relu', kernel_regularizer=l2(l2_lambda))(input_phrase2)

    combined = concatenate([fcnn_en1, fcnn_en2])
    dropout = Dropout(dropout_rate)(combined)

    classification = Dense(1, activation='tanh')(dropout)

    model = Model(inputs=[input_phrase1, input_phrase2], outputs=classification)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

embedding_dim = 300
fcnn_units = 128
dropout_rate = 0.5
l2_lambda = 0.001

model = create_binary_classification_model(embedding_dim, fcnn_units, dropout_rate, l2_lambda)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    [s1_train, s2_train], yfcnn_train,
    epochs=50,
    batch_size=32,
    validation_data=([s1_validate, s2_validate], yfcnn_validate),
    callbacks=[early_stopping])
model.save("model.h5")

test_loss, test_accuracy = model.evaluate([s1_test, s2_test], yfcnn_test)
predictions = model.predict([s1_test, s2_test])

thresholded_predictions = (predictions > 0.5).astype(int)
print(classification_report(yfcnn_test, thresholded_predictions))
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/50
219/219 [==============================] - 3s 6ms/step - loss: 0.9594 - accuracy: 0.5379 - val_loss: 0.8420 - val_accuracy: 0.5550
Epoch 2/50
219/219 [==============================] - 2s 8ms/step - loss: 0.8023 - accuracy: 0.5479 - val_loss: 0.7722 - val_accuracy: 0.5210
Epoch 3/50
219/219 [==============================] - 1s 7ms/step - loss: 0.7491 - accuracy: 0.5500 - val_loss: 0.7370 - val_accuracy: 0.5380
Epoch 4/50
219/219 [==============================] - 1s 6ms/step - loss: 0.7237 - accuracy: 0.5549 - val_loss: 0.7197 - val_accuracy: 0.5530
Epoch 5/50
219/219 [==============================] - 1s 5ms/step - loss: 0.7113 - accuracy: 0.5539 - val_loss: 0.7157 - val_accuracy: 0.5540
Epoch 6/50
219/219 [==============================] - 1s 5ms/step - loss: 0.7040 - accuracy: 0.5574 - val_loss: 0.7053 - val_accuracy: 0.5460
Epoch 7/50
219/219 [==============================] - 1s 5ms/step - loss: 0.7004 - accuracy: 0.5560 - val_loss: 0.7034 - val_accuracy: 0.5270
Epoch 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


63/63 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.57      0.84      0.68      1111
           1       0.52      0.22      0.31       889

    accuracy                           0.56      2000
   macro avg       0.55      0.53      0.50      2000
weighted avg       0.55      0.56      0.52      2000

Test Loss: 0.6900848746299744
Test Accuracy: 0.5644999742507935


## Bonus Task: BERT FINETUNING

### Sentence

In [ ]:
!pip install bert_score
from bert_score import score

predicted_paraphrases = test_df[(thresholded_predictions == 1)]
predicted_different = test_df[(thresholded_predictions != 1)]
P, R, F1 = score(predicted_paraphrases['sentence1'].tolist(), predicted_paraphrases['sentence2'].tolist(), lang="en")
# P2, R2, F1_2 = score(predicted_different['sentence1'].tolist(), predicted_different['sentence2'].tolist(), lang = 'en')
P_mean = P.mean().item()
R_mean = R.mean().item()
F1_mean = F1.mean().item()
# P2_mean = P2.mean().item()
# R2_mean = R2.mean().item()
# F1_2_mean = F1_2.mean().item()
print(f"BERTScore Precision (mean) for predicted paraphrases: {P_mean}")
print(f"BERTScore Recall (mean) for predicted paraphrases: {R_mean}")
print(f"BERTScore F1 (mean) for predicted paraphrases: {F1_mean}")



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision (mean) for predicted paraphrases: 0.9697710871696472
BERTScore Recall (mean) for predicted paraphrases: 0.9699026942253113
BERTScore F1 (mean) for predicted paraphrases: 0.9698091745376587


In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, InputExample, losses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.5 MB/s eta 0:00:00


In [ ]:
sentence1_tr = train_df['sentence1'].tolist()
sentence2_tr = train_df['sentence2'].tolist()
sentence1_tes = test_df['sentence1'].tolist()
sentence2_tes = test_df['sentence2'].tolist()
labels = train_df['label'].tolist()


sentence_bert_model = SentenceTransformer('bert-base-nli-mean-tokens')
train_examples = [InputExample(texts=[s1, s2], label=label) for s1, s2, label in zip(sentence1_tr, sentence2_tr, labels)]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.ContrastiveLoss(model=sentence_bert_model)
#contrasive loss calculates similarity/dissimilarity between samples

sentence_bert_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)


NameError: name 'train_df' is not defined

In [ ]:

print("encoding the test data.....")
sentence1_embeddings = sentence_bert_model.encode(sentence1_tes, convert_to_tensor=True)
sentence2_embeddings = sentence_bert_model.encode(sentence2_tes, convert_to_tensor=True)


saving the model......


NameError: name 'sentence_bert_model' is not defined

In [ ]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

similarity_scores = []

for index in range(len(test_df)):
    sim_score = cosine_similarity(sentence1_embeddings[index], sentence2_embeddings[index])
    similarity = 1 if sim_score > 0.5 else 0
    similarity_scores.append(similarity)

test_df['similarity'] = similarity_scores

accuracy = (test_df['similarity'] == test_df['label']).mean()

print(f"Accuracy based on cosine similarity threshold: {accuracy * 100:.2f}%")


Accuracy based on cosine similarity threshold: 52.20%


### Phrase 

In [ ]:

dataset = load_dataset("PiC/phrase_similarity")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 7004
    })
    validation: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['phrase1', 'phrase2', 'sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2000
    })
})

In [ ]:
train_dataset_p = dataset['train']
validation_dataset_p = dataset['validation']
test_dataset_p = dataset['test']
train_df_p = train_dataset_p.to_pandas()
validation_df_p = validation_dataset_p.to_pandas()
test_df_p = test_dataset_p.to_pandas()
phrase1_tr = train_df_p['phrase1'].tolist()
phrase2_tr = train_df_p['phrase2'].tolist()
phrase1_tes = test_df_p['phrase1'].tolist()
phrase2_tes = test_df_p['phrase2'].tolist()
labels_p = train_df_p['label'].tolist()

sentence_bert_model_p = SentenceTransformer('bert-base-nli-mean-tokens')

train_examples_p = [InputExample(texts=[s1, s2], label=label) for s1, s2, label in zip(phrase1_tr, phrase2_tr, labels_p)]
train_dataloader_p = DataLoader(train_examples_p, shuffle=True, batch_size=16)

train_loss_p = losses.ContrastiveLoss(model=sentence_bert_model_p)
sentence_bert_model_p.fit(train_objectives=[(train_dataloader_p, train_loss_p)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/438 [00:00<?, ?it/s]

Iteration:   0%|          | 0/438 [00:00<?, ?it/s]

Iteration:   0%|          | 0/438 [00:00<?, ?it/s]

Iteration:   0%|          | 0/438 [00:00<?, ?it/s]

Iteration:   0%|          | 0/438 [00:00<?, ?it/s]

In [ ]:
print("saving the model......")
model_save_path = 'fine_tuned_sentence_bert_phrase'
sentence_bert_model_p.save(model_save_path)

print("encoding the test data.....")
phrase1_embeddings = sentence_bert_model_p.encode(phrase1_tes, convert_to_tensor=True)
phrase2_embeddings = sentence_bert_model_p.encode(phrase2_tes, convert_to_tensor=True)


saving the model......
encoding the test data.....


In [ ]:

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

similarity_scores = []

for index in range(len(test_df_p)):
    sim_score = cosine_similarity(phrase1_embeddings[index], phrase2_embeddings[index])
    similarity = 1 if sim_score > 0.5 else 0
    similarity_scores.append(similarity)

test_df_p['similarity'] = similarity_scores

accuracy = (test_df_p['similarity'] == test_df_p['label']).mean()

print(f"Accuracy based on cosine similarity threshold: {accuracy * 100:.2f}%")

Accuracy based on cosine similarity threshold: 49.95%
